# TCN Classification
<hr>




## Load the library

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  del sys.path[0]


In [5]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the Dataset

In [6]:
corpus = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentence_version_1.csv')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(8679, 4)


,Unnamed: 0,sentence,label,split
0,1,van huy,0,train
1,2,end intern,0,train
2,3,education,0,train
3,4,vietnam korea university information communica...,1,train
4,5,project school website sell bakery,0,train
...,...,...,...,...
8674,8675,programming tool mysql workbench intellij ide,1,train
8675,8676,coffee shop project management,1,train
8676,8677,description manage coffee shop activity add in...,1,train
8677,8678,technology run time environment microsoft wind...,1,train


In [ ]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8679 entries, 0 to 8678
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8679 non-null   int64 
 1   sentence    8679 non-null   object
 2   label       8679 non-null   int64 
 3   split       8679 non-null   object
dtypes: int64(2), object(2)
memory usage: 271.3+ KB


In [ ]:
corpus.groupby(by=['split', 'label']).count()

Unnamed: 0  sentence
split label                      
test  0            1057      1057
      1             743       743
train 0            3945      3945
      1            2934      2934

In [7]:
# Separate the sentences and the labels
# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print(len(train_x))
print(len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print(len(test_x))
print(len(test_y))

6879
6879
1800
1800


In [8]:
train_x[1000]

'time management skill'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [15]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

print("Example of sentence: ", train_x[70])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[70])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[70])

Example of sentence:  company fpt softwate test manual 10/2019
Into a sequence of int: [30, 112, 2037, 8, 235, 181, 71]
Into a padded sequence: [  30  112 2037    8  235  181   71    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    

In [17]:
# See the first 10 words in the vocabulary

word_index = tokenizer.word_index
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
project 2
technology 3
management 4
web 5
experience 6
java 7
test 8
work 9
html 10
4321


# Model 1: Embedding Random
<hr>

## TCN Model

Now, we will build Temporal Convolutional Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from https://github.com/philipperemy/keras-tcn and https://www.kaggle.com/christofhenkel/temporal-convolutional-network

__Arguments__
`TCN(nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=False, dropout_rate=0.0, return_sequences=True, activation='relu', kernel_initializer='he_normal', use_batch_norm=False, **kwargs)`

- `nb_filters`: Integer. The number of filters to use in the convolutional layers. Would be similar to units for LSTM.
- `kernel_size`: Integer. The size of the kernel to use in each convolutional layer.
- `dilations`: List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
- `nb_stacks`: Integer. The number of stacks of residual blocks to use.
- `padding`: String. The padding to use in the convolutions. 'causal' for a causal network (as in the original implementation) and - `'same' for a non-causal network.
- `use_skip_connections`: Boolean. If we want to add skip connections from input to each residual block.
- `return_sequences`: Boolean. Whether to return the last output in the output sequence, or the full sequence.
- `dropout_rate`: Float between 0 and 1. Fraction of the input units to drop.
- `activation`: The activation used in the residual blocks o = activation(x + F(x)).
- `kernel_initializer`: Initializer for the kernel weights matrix (Conv1D).
- `use_batch_norm`: Whether to use batch normalization in the residual layers or not.
- `kwargs`: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model. Use unique names when using multiple TCN.

Now, we will define our TCN model as follows:
- One TCN layer with 100 filters, kernel size 1-6, and relu and tanh activation function;
- Dropout size = 0.5;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [25]:
!pip install keras-tcn

In [ ]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [ ]:
model_0 = define_model(input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     400256      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

In [ ]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') > 0.93):
#             print("\nReached 93% accuracy so cancelling training!")
#             self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/driver')
nb_path = '/content/notebooks'

from google.colab import drive
drive.mount('/content/drive')

# Install only once. Tomorrow, you can skip this.
!pip install --target=$nb_path helpers

Mounted at /content/driver
Mounted at /content/drive


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1,2,3,4,5,6]

columns = ['Activation', 'Filters', 'Acc']
record = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0
acc_max = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        # Define the input shape
        model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.7):

            history = model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            else:
                # # plot_train_history(history, x_ticks_vertical=True)
                # # summarize history for accuracy
                # plt.plot(history.history['accuracy'])
                # plt.plot(history.history['val_accuracy'])
                # plt.title('model accuracy')
                # plt.ylabel('accuracy')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                # # summarize history for loss
                # plt.plot(history.history['loss'])
                # plt.plot(history.history['val_loss'])
                # plt.title('model loss')
                # plt.ylabel('loss')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()

                # y_pred = model.predict(Xtest)
                # print(confusion_matrix(test_y,y_pred))
                # print(classification_report(test_y,y_pred))
                # print(accuracy_score(test_y, y_pred))

                if acc > acc_max:
                    acc_max = acc
                    model.save_weights('/content/drive/MyDrive/HR_Akatesuki/model/model_best_version1.h5')
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

print('model co do chinh xac cao nhat:', acc_max)

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/100
138/138 [==============================] - 8s 42ms/step - loss: 0.4585 - accuracy: 0.7747 - val_loss: 0.2293 - val_accuracy: 0.9194
Epoch 2/100
138/138 [==============================] - 5s 36ms/step - loss: 0.2551 - accuracy: 0.9033 - val_loss: 0.2506 - val_accuracy: 0.9006
Epoch 3/100
138/138 [==============================] - 5s 37ms/step - loss: 0.1939 - accuracy: 0.9246 - val_loss: 0.2861 - val_accuracy: 0.8806
Epoch 4/100
138/138 [==============================] - 5s 37ms/step - loss: 0.1550 - accuracy: 0.9427 - val_loss: 0.3382 - val_accuracy: 0.8917
Epoch 5/100
138/138 [==============================] - 5s 36ms/step - loss: 0.1122 - accuracy: 0.9641 - val_loss: 0.4612 - val_accuracy: 0.8661
Epoch 6/100
138/138 [==============================] - 5s 37ms/step - loss: 0.0823 - accuracy: 0.9719 - val_loss: 0.4882 - val_accuracy: 0.8800
Epoc

## Summary

In [ ]:
record.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
6,tanh,1,0.925556
7,tanh,2,0.922222
9,tanh,4,0.921667
0,relu,1,0.919444
4,relu,5,0.918333
2,relu,3,0.915556
11,tanh,6,0.908889
1,relu,2,0.907778
8,tanh,3,0.907222
5,relu,6,0.905000


In [ ]:
report = record.sort_values(by='Acc', ascending=False)
report = report.to_excel('/content/drive/MyDrive/HR_Akatesuki/report_version_1/TCN_version_1-model1.xlsx', sheet_name='random')

## Test model best

In [ ]:
# load CV test
CV_test_path = '/content/drive/MyDrive/HR_Akatesuki/[CV_test][Fresher]BuiHoangNhat-PHP.csv'
df = pd.read_csv(CV_test_path)
test_cv = list(df.Sentence)

# processing
test_sequences = tokenizer.texts_to_sequences(test_cv)
max_len = max_length(training_sequences)
CVtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

# Load best model
model.load_weights('/content/drive/MyDrive/HR_Akatesuki/model/model_best_version1.h5')

# predict CV_test
predict = model.predict(CVtest, verbose=0)
yhat_probs = np.argmax(predict, axis=1)
predict_data = pd.DataFrame(predict , columns=["U", "K"])

# save resuilt
predict_data.to_csv('/content/drive/MyDrive/HR_Akatesuki/report_version_1/[Predict]BuiHoangNhat-PHP.csv')
df.head(10)

,Sentence
0,bui hoang nhat
1,apply fresher php
2,gpa 9.0/10
3,grade excellent
4,july 20 1998
5,male
6,0932435066
7,nhatbui2017@gmail.com
8,duy hai duy xuyen quang nam
9,facebook.com/nhat.hoangbui.982


In [ ]:
# resuil predict
yhat_probs

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0])

In [ ]:
#  show view result
count = 0
for i in range(len(test_cv)):  
    if (yhat_probs[i]==0):
        print(test_cv[i],' ---------> Unknow')     
    elif (yhat_probs[i]==1):
        count += 1
        print(test_cv[i],' ---------> Knowleadge') 
print('Dem duoc: ',count ,' tu thuoc Knowleaged')

bui hoang nhat  ---------> Unknow
apply  fresher php  ---------> Knowleadge
gpa 9.0/10  ---------> Unknow
grade excellent  ---------> Unknow
july 20 1998  ---------> Unknow
male  ---------> Unknow
0932435066  ---------> Unknow
nhatbui2017@gmail.com  ---------> Unknow
duy hai duy xuyen quang nam  ---------> Unknow
facebook.com/nhat.hoangbui.982  ---------> Unknow
/  ---------> Unknow
career objective  ---------> Unknow
enter  profession   passion  ---------> Unknow
programming   expertise  ---------> Unknow
learn   iviettech certificate  ---------> Unknow
want    php programmer  ---------> Knowleadge
able    good use  ---------> Unknow
ability hopefully  work  ---------> Unknow
environment   company  help  ---------> Unknow
equip  experience  skill  ---------> Knowleadge
achieve  goal  ---------> Unknow
good programmer   future  ---------> Unknow
english skill  ---------> Knowleadge
good  read  research  ---------> Knowleadge
material  ---------> Unknow
hobby  ---------> Unknow
music  -

In [ ]:
# Knowleadge predict
knowledge = ''
for i in range(len(test_cv)):
  if (yhat_probs[i]==1):
    knowledge += ' ' + test_cv[i]

print(knowledge)

 apply  fresher php want    php programmer equip  experience  skill english skill good  read  research university  danang university  education major php programmer technical skill php web developer php develop application   require  database phpmyadmin  work   mysql database develop php application   datum   store datum  mysql database analyze  design database develop  project  laravel framework php html css javascript bootstrap framework laravel mysql sql server website  manage online sale  electronic item responsibility analyze  design database build website interface build basic programming language php database mysql technology  laravel framework 8.1 experience   jquery javascript work  mysql work  laravel framework experience   html css  bootstrap build  sale management website description  website    da nang pedagogical university  base   idea responsibility database analysis  design programming language php database mysql gain experience   php programming language combine html 

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [9]:
!brew install wget

!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

/bin/bash: brew: command not found
--2021-08-06 18:47:27--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.204.40
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.204.40|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [10]:
!gzip -d GoogleNews-vectors-negative300.bin.gzy

gzip: GoogleNews-vectors-negative300.bin.gzy.gz: No such file or directory


In [11]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
# word2vec.word_vec('framework') #-0.13085938
word2vec.word_vec('python')  # 0.27734375

2. __Check number of training words present in Word2Vec__

In [12]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [18]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 2983 words present from 4915 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [19]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

In [20]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [21]:
# Test the function
w_2_i = {'<UNK>': 1, 'framework': 2, 'java': 3, 'python': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [-0.13085938,  0.03881836,  0.07177734, ..., -0.07080078,
        -0.07324219,  0.00463867],
       [-0.13574219, -0.19042969, -0.37304688, ...,  0.01867676,
         0.06542969,  0.29882812],
       [ 0.27734375,  0.12402344, -0.40820312, ...,  0.49023438,
        -0.21679688, -0.00622559]])

In [22]:
em_matrix.shape

(5, 300)

## TCN Model

In [27]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model_2(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [28]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     400256      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

## Train and Test the Model

In [29]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') >= 0.9):
#             print("\nReached 90% accuracy so cancelling training!")
#             self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [30]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1,2,3,4,5,6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'Acc']
record2 = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0
acc_max = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

        # Define the input shape
        model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                             max_length=max_len, emb_matrix=emb_matrix)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.6):
            history = model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                                       max_length=max_len, emb_matrix=emb_matrix)
            else:
                # # summarize history for accuracy
                # plt.plot(history.history['accuracy'])
                # plt.plot(history.history['val_accuracy'])
                # plt.title('model accuracy')
                # plt.ylabel('accuracy')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                # # summarize history for loss
                # plt.plot(history.history['loss'])
                # plt.plot(history.history['val_loss'])
                # plt.title('model loss')
                # plt.ylabel('loss')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                if acc > acc_max:
                    acc_max = acc
                    model.save_weights('/content/drive/MyDrive/HR_Akatesuki/model/model-2_best_version1.h5')
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

print('model co do chinh xac cao nhat:', acc_max)

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/100
138/138 [==============================] - 39s 37ms/step - loss: 0.5246 - accuracy: 0.7343 - val_loss: 0.4069 - val_accuracy: 0.8278
Epoch 2/100
138/138 [==============================] - 4s 27ms/step - loss: 0.3661 - accuracy: 0.8372 - val_loss: 0.2739 - val_accuracy: 0.8917
Epoch 3/100
138/138 [==============================] - 3s 25ms/step - loss: 0.2982 - accuracy: 0.8741 - val_loss: 0.2081 - val_accuracy: 0.9200
Epoch 4/100
138/138 [==============================] - 3s 24ms/step - loss: 0.2520 - accuracy: 0.8988 - val_loss: 0.2437 - val_accuracy: 0.9061
Epoch 5/100
138/138 [==============================] - 4s 27ms/step - loss: 0.2107 - accuracy: 0.9193 - val_loss: 0.1923 - val_accuracy: 0.9244
Epoch 6/100
138/138 [==============================] - 4s 27ms/step - loss: 0.1755 - accuracy: 0.9328 - val_loss: 0.2074 - val_accuracy: 0.9222
Epo

## Summary

In [31]:
record2.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
0,relu,1,0.924444
4,relu,5,0.922222
3,relu,4,0.920000
5,relu,6,0.917778
2,relu,3,0.915556
1,relu,2,0.913889
9,tanh,4,0.907778
7,tanh,2,0.902778
10,tanh,5,0.900556
8,tanh,3,0.896667


In [32]:
report = record2.sort_values(by='Acc', ascending=False)
report = report.to_excel('/content/drive/MyDrive/HR_Akatesuki/report_version_1/TCN_version_1-model2.xlsx', sheet_name='static')

## Test model 2


In [34]:
# load CV test
CV_test_path = '/content/drive/MyDrive/HR_Akatesuki/[CV_test][Fresher]BuiHoangNhat-PHP.csv'
df = pd.read_csv(CV_test_path)
test_cv = list(df.Sentence)

# processing
test_sequences = tokenizer.texts_to_sequences(test_cv)
max_len = max_length(training_sequences)
CVtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

# Load best model
model.load_weights('/content/drive/MyDrive/HR_Akatesuki/model/model-2_best_version1.h5')

# predict CV_test
predict = model.predict(CVtest, verbose=0)
yhat_probs = np.argmax(predict, axis=1)
predict_data = pd.DataFrame(predict , columns=["U", "K"])

# save resuilt
predict_data.to_csv('/content/drive/MyDrive/HR_Akatesuki/report_version_1/[Predict]BuiHoangNhat-PHP-model2-ver1.csv')
df.head(10)

,Sentence
0,bui hoang nhat
1,apply fresher php
2,gpa 9.0/10
3,grade excellent
4,july 20 1998
5,male
6,0932435066
7,nhatbui2017@gmail.com
8,duy hai duy xuyen quang nam
9,facebook.com/nhat.hoangbui.982


In [35]:
# resuil predict
yhat_probs

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0])

In [36]:
#  show view result
count = 0
for i in range(len(test_cv)):  
    if (yhat_probs[i]==0):
        print(test_cv[i],' ---------> Unknow')     
    elif (yhat_probs[i]==1):
        count += 1
        print(test_cv[i],' ---------> Knowleadge') 
print('Dem duoc: ',count ,' tu thuoc Knowleaged')

bui hoang nhat  ---------> Unknow
apply  fresher php  ---------> Unknow
gpa 9.0/10  ---------> Unknow
grade excellent  ---------> Unknow
july 20 1998  ---------> Unknow
male  ---------> Unknow
0932435066  ---------> Unknow
nhatbui2017@gmail.com  ---------> Unknow
duy hai duy xuyen quang nam  ---------> Unknow
facebook.com/nhat.hoangbui.982  ---------> Unknow
/  ---------> Unknow
career objective  ---------> Unknow
enter  profession   passion  ---------> Unknow
programming   expertise  ---------> Knowleadge
learn   iviettech certificate  ---------> Unknow
want    php programmer  ---------> Knowleadge
able    good use  ---------> Unknow
ability hopefully  work  ---------> Unknow
environment   company  help  ---------> Unknow
equip  experience  skill  ---------> Unknow
achieve  goal  ---------> Unknow
good programmer   future  ---------> Unknow
english skill  ---------> Knowleadge
good  read  research  ---------> Unknow
material  ---------> Unknow
hobby  ---------> Knowleadge
music  -----

In [37]:
# Knowleadge predict
knowledge = ''
for i in range(len(test_cv)):
  if (yhat_probs[i]==1):
    knowledge += ' ' + test_cv[i]

print(knowledge)

 programming   expertise want    php programmer english skill hobby major php programmer technical skill php web developer php develop application   require  database phpmyadmin  work   mysql database develop php application   datum   store datum  mysql database analyze  design database develop  project  laravel framework programming php html css javascript bootstrap framework laravel database mysql sql server description     project   implement   iviettech center build  sale responsibility analyze  design database build website interface build basic programming language php database mysql technology  laravel framework 8.1 database design  analysis experience   jquery javascript work  mysql work  laravel framework experience   html css  bootstrap responsibility database analysis  design programming language php database mysql learn   build  database gain experience   php programming language combine html css javascript effectively soft skill ability  solve problem good teamwork ability

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## TCN Model


In [42]:
def define_model_3(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = True)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="softmax")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [43]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 100, 300)     300000      input_16[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_15 (SpatialDr (None, 100, 300)     0           embedding_15[0][0]               
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     400256      spatial_dropout1d_15[0][0]       
___________________________________________________________________________________________

## Train and Test the Model

In [44]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [45]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1,2,3,4,5,6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'Acc']
record3 = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

exp = 0
acc_max = 0

for activation in activations:

    for kernel_size in kernel_sizes:
        
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

        # Define the input shape
        model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                             max_length=max_len, emb_matrix=emb_matrix)

        # Train the model and initialize test accuracy with 0
        acc = 0
        while(acc<0.6):

            model.fit(Xtrain, train_y, batch_size=50, epochs=200, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            if (acc<0.6):
                print('The model suffered from local minimum. Retrain the model!')
                model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                                       max_length=max_len, emb_matrix=emb_matrix)
            else:
                # # summarize history for accuracy
                # plt.plot(history.history['accuracy'])
                # plt.plot(history.history['val_accuracy'])
                # plt.title('model accuracy')
                # plt.ylabel('accuracy')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                # # summarize history for loss
                # plt.plot(history.history['loss'])
                # plt.plot(history.history['val_loss'])
                # plt.title('model loss')
                # plt.ylabel('loss')
                # plt.xlabel('epoch')
                # plt.legend(['train', 'test'], loc='upper left')
                # plt.show()
                if acc > acc_max:
                    acc_max = acc
                    model.save_weights('/content/drive/MyDrive/HR_Akatesuki/model/model-3_best_version1.h5')
                print('Done!')

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()
        print('model co do chinh xac cao nhat:', acc_max)

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/200
138/138 [==============================] - 8s 44ms/step - loss: 0.5345 - accuracy: 0.7469 - val_loss: 0.2755 - val_accuracy: 0.8961
Epoch 2/200
138/138 [==============================] - 7s 51ms/step - loss: 0.2981 - accuracy: 0.8775 - val_loss: 0.2241 - val_accuracy: 0.9078
Epoch 3/200
138/138 [==============================] - 5s 40ms/step - loss: 0.2239 - accuracy: 0.9131 - val_loss: 0.2161 - val_accuracy: 0.9106
Epoch 4/200
138/138 [==============================] - 7s 51ms/step - loss: 0.1689 - accuracy: 0.9353 - val_loss: 0.2524 - val_accuracy: 0.9011
Epoch 5/200
138/138 [==============================] - 6s 47ms/step - loss: 0.1367 - accuracy: 0.9466 - val_loss: 0.2674 - val_accuracy: 0.8961
Epoch 6/200
138/138 [==============================] - 7s 51ms/step - loss: 0.1001 - accuracy: 0.9635 - val_loss: 0.3585 - val_accuracy: 0.8883
Epoc

## Summary

In [46]:
record3.sort_values(by='Acc', ascending=False)

,Activation,Filters,Acc
2,relu,3,0.924444
5,relu,6,0.923333
3,relu,4,0.915556
4,relu,5,0.915000
1,relu,2,0.912778
0,relu,1,0.910556


In [47]:
report = record3.sort_values(by='Acc', ascending=False)
report = report.to_excel('/content/drive/MyDrive/HR_Akatesuki/report_version_1/TCN_version_1-3.xlsx', sheet_name='dynamic')

## Test model 3

In [49]:
# load CV test
CV_test_path = '/content/drive/MyDrive/HR_Akatesuki/[CV_test][Fresher]BuiHoangNhat-PHP.csv'
df = pd.read_csv(CV_test_path)
test_cv = list(df.Sentence)

# processing
test_sequences = tokenizer.texts_to_sequences(test_cv)
max_len = max_length(training_sequences)
CVtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

# Load best model
model.load_weights('/content/drive/MyDrive/HR_Akatesuki/model/model-3_best_version1.h5')

# predict CV_test
predict = model.predict(CVtest, verbose=0)
yhat_probs = np.argmax(predict, axis=1)
predict_data = pd.DataFrame(predict , columns=["U", "K"])

# save resuilt
predict_data.to_csv('/content/drive/MyDrive/HR_Akatesuki/report_version_1/[Predict]BuiHoangNhat-PHP-model3-ver1.csv')
df.head(10)

,Sentence
0,bui hoang nhat
1,apply fresher php
2,gpa 9.0/10
3,grade excellent
4,july 20 1998
5,male
6,0932435066
7,nhatbui2017@gmail.com
8,duy hai duy xuyen quang nam
9,facebook.com/nhat.hoangbui.982


In [50]:
#  show view result
count = 0
for i in range(len(test_cv)):  
    if (yhat_probs[i]==0):
        print(test_cv[i],' ---------> Unknow')     
    elif (yhat_probs[i]==1):
        count += 1
        print(test_cv[i],' ---------> Knowleadge') 
print('Dem duoc: ',count ,' tu thuoc Knowleaged')

bui hoang nhat  ---------> Unknow
apply  fresher php  ---------> Knowleadge
gpa 9.0/10  ---------> Unknow
grade excellent  ---------> Unknow
july 20 1998  ---------> Unknow
male  ---------> Unknow
0932435066  ---------> Unknow
nhatbui2017@gmail.com  ---------> Unknow
duy hai duy xuyen quang nam  ---------> Unknow
facebook.com/nhat.hoangbui.982  ---------> Unknow
/  ---------> Unknow
career objective  ---------> Unknow
enter  profession   passion  ---------> Unknow
programming   expertise  ---------> Unknow
learn   iviettech certificate  ---------> Unknow
want    php programmer  ---------> Knowleadge
able    good use  ---------> Unknow
ability hopefully  work  ---------> Unknow
environment   company  help  ---------> Unknow
equip  experience  skill  ---------> Knowleadge
achieve  goal  ---------> Unknow
good programmer   future  ---------> Unknow
english skill  ---------> Knowleadge
good  read  research  ---------> Knowleadge
material  ---------> Unknow
hobby  ---------> Unknow
music  -

In [51]:
# Knowleadge predict
knowledge = ''
for i in range(len(test_cv)):
  if (yhat_probs[i]==1):
    knowledge += ' ' + test_cv[i]

print(knowledge)

 apply  fresher php want    php programmer equip  experience  skill english skill good  read  research major php programmer technical skill php web developer php develop application   require  database phpmyadmin  work   mysql database develop php application   datum   store datum  mysql database develop  project  laravel framework php html css javascript bootstrap framework laravel mysql sql server responsibility analyze  design database build website interface build basic programming language php database mysql technology  laravel framework 8.1 gain experience  skill  project experience   jquery javascript work  mysql work  laravel framework experience   html css  bootstrap programming language php database mysql gain experience  skill  project gain experience   php programming language combine html css javascript effectively good teamwork ability
